In [9]:
# !pip install pyarrow
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
df = pd.read_parquet("/Users/masha/Downloads/stackoverflow_sample.parquet")
df.head()

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditDate,...,Title,Tags,AnswerCount,CommentCount,ContentLicense,FavoriteCount,ClosedDate,OwnerDisplayName,LastEditorDisplayName,CommunityOwnedDate
0,65528592,1,65528859,2021-01-01T09:30:19.827,-1,1471,<p>I am using SQL Server.</p>\n<p>My table is ...,1492229,1127428,2021-01-02T05:56:05.060,...,How to select all parents of a child from a table,|sql|sql-server|,2,3,CC BY-SA 4.0,None,None,None,None,None
1,65528597,1,65558664,2021-01-01T09:31:22.163,1,974,<p>I am trying to use <code>Azure Stream Analy...,8074614,14884454,2021-01-01T09:35:39.870,...,Generating unique identifier from Azure Stream...,|azure-stream-analytics|,1,0,CC BY-SA 4.0,None,None,None,None,None
2,65528599,1,None,2021-01-01T09:31:28.293,1,2091,<p>This is my Vuex getter:</p>\n<pre><code>con...,14062042,63550,2023-01-09T22:17:14.270,...,How can I use Vuex modules getters with compos...,|javascript|vue.js|vuex|vuejs3|,2,0,CC BY-SA 4.0,None,None,None,None,None
3,65528603,1,65528811,2021-01-01T09:32:39.457,0,211,<pre><code>import tkinter as tk\nimport thread...,5204228,None,None,...,Python Tkinter variable not updating in other ...,|python|tkinter|python-multithreading|,1,2,CC BY-SA 4.0,None,None,None,None,None
4,65528608,1,65539277,2021-01-01T09:33:40.527,0,379,"<p>For some strange reason, the option to disa...",6904665,None,None,...,Disable plugin not present in Jenkins,|jenkins|groovy|jenkins-plugins|devops|,1,3,CC BY-SA 4.0,None,None,None,None,None


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()  # removes HTML elements
    text = text.lower()  # lower-casing
    words = word_tokenize(text)  # tokenize
    words = [word for word in words if word.isalnum()]  
    words = [word for word in words if word not in stop_words] 
    stemmed_text = " ".join([stemmer.stem(word) for word in words])  # stemming
    return stemmed_text

df_cleaned = df[['Body']].copy()
df_cleaned['cleaned_body'] = df_cleaned['Body'].apply(preprocess_text)

df_cleaned.head()

[nltk_data] Downloading package punkt to /Users/masha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/masha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Body,cleaned_body
0,<p>I am using SQL Server.</p>\n<p>My table is ...,use sql server tabl call employe look like emp...
1,<p>I am trying to use <code>Azure Stream Analy...,tri use azur stream analyt job write data sql ...
2,<p>This is my Vuex getter:</p>\n<pre><code>con...,vuex getter const getter getmoviebyid state id...
3,<pre><code>import tkinter as tk\nimport thread...,import tkinter tk import thread import import ...
4,"<p>For some strange reason, the option to disa...",strang reason option disabl plugin present jen...


In [ ]:
df_cleaned.to_parquet("cleaned_data.parquet", index=False)